In [ ]:
! pip install mistralai
! pip install langchain
! pip install langchain-community langchain-core
! pip install -U langchain-mistralai
! pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.6 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores.chroma import Chroma

In [ ]:
# add google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/

/content/drive/My Drive


In [ ]:
# Load data
loader = TextLoader("mistral/nvidia_ocp_docs.txt")
docs = loader.load()

In [ ]:
# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

In [ ]:
documents

[Document(page_content="[Document(page_content='NVIDIA GPU  architecture overview\\nNVIDIA GPU prerequisites\\nNVIDIA GPU enablement\\nGPUs and bare metal\\nGPUs and virtualization\\nGPUs and vSphere\\nGPUs and Red Hat KVM\\nGPUs and CSPs\\nGPUs and Red Hat Device Edge\\nGPU sharing methods\\nCUDA streams\\nTime-slicing\\nCUDA Multi-Process Service\\nMulti-instance GPU\\nVirtualization with vGPU\\nNVIDIA GPU features for OpenShift Container Platform\\nNVIDIA supports the use of graphics processing unit (GPU) resources on OpenShift', metadata={'source': 'mistral/nvidia_ocp_414.pdf', 'page': 0}),\n Document(page_content='Container Platform. OpenShift Container Platform is a security-focused and\\nhardened Kubernetes platform developed and supported by Red Hat for deploying\\nand managing Kubernetes clusters at scale. OpenShift Container Platform includes\\nenhancements to Kubernetes so that users can easily configure and use NVIDIA\\nGPU resources to accelerate workloads.\\nThe NVIDIA GP

In [ ]:
# Define the embedding model
embeddings = MistralAIEmbeddings(model="mistral-embed", mistral_api_key="your_secret_key")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_mistralai/embeddings.py:105: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [ ]:
# Create the vector store
vector = Chroma.from_documents(documents, embeddings)

In [ ]:
# Define a retriever interface
retriever = vector.as_retriever()

In [ ]:
# Define LLM
model = ChatMistralAI(mistral_api_key="your_private_key")

In [ ]:

# Define prompt template
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "What is MIG"})
print(response["answer"])

MIG stands for Multi-Instance GPU, which is a feature of the new NVIDIA Ampere architecture. It enables splitting GPU compute units and memory into multiple MIG instances, each of which represents a standalone GPU device from a system perspective and can be connected to any application, container, or virtual machine running on the node. The software that uses the GPU treats each of these MIG instances as an individual GPU. This feature is useful when you have an application that does not require the full power of an entire GPU. The NVIDIA GPU Operator version 1.7.0 and higher provides MIG support for the A100 and A30 Ampere cards.


In [ ]:
# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "What Nvidia GPU models are supported by MIG in OpenShift 4.14?"})
print(response["answer"])

The NVIDIA GPU models that are supported by MIG (Multi-Instance GPU) in OpenShift 4.14 are A30, A100, A100X, A800, AX800, H100, and H800.
